In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 70)
pd.set_option('display.max_columns', 50)

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from google.cloud import bigquery
from google.oauth2 import service_account
from pathlib import Path
from sklearn import set_config; set_config(display='diagram')
from sklearn.compose import ColumnTransformer, make_column_transformer

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn import set_config; set_config(display='diagram')
from sklearn.preprocessing import FunctionTransformer
from flaml import AutoML



In [4]:
df = pd.read_csv('../../data/clean_data_final.csv')
df

,TRAIN_SERVICE_CODE_AFFECTED,SERVICE_GROUP_CODE_AFFECTED,ENGLISH_DAY_TYPE,APP_TIMETABLE_FLAG_AFF,UNIT_CLASS_AFFECTED,INCIDENT_REASON,PERFORMANCE_EVENT_CODE,PFPI_MINUTES,Lat_OR,Lon_OR,Lat_DES,Lon_DES,ORIG_MONTH_SIN,ORIG_MONTH_COS,ORIG_DAY_SIN,ORIG_DAY_COS,ORIG_HOUR_SIN,ORIG_HOUR_COS,ORIG_MINUTE_SIN,ORIG_MINUTE_COS,DEST_MONTH_SIN,DEST_MONTH_COS,DEST_DAY_SIN,DEST_DAY_COS,DEST_HOUR_SIN,DEST_HOUR_COS,DEST_MINUTE_SIN,DEST_MINUTE_COS
0,22215003,EK03,BH,Y,375.0,M,M,9.0,51.54650,-0.10408,51.54650,-0.10408,0.406737,0.913545,0.207912,0.978148,0.743145,-0.669131,-0.309017,0.951057,0.406737,0.913545,0.207912,0.978148,0.669131,-0.743145,-9.781476e-01,0.207912
1,22215003,EK03,BH,Y,375.0,M,M,6.0,51.54650,-0.10408,51.54612,-0.07513,0.406737,0.913545,0.207912,0.978148,0.994522,-0.104528,-0.951057,-0.309017,0.406737,0.913545,0.207912,0.978148,0.978148,-0.207912,-2.079117e-01,-0.978148
2,22218000,EK99,BH,Y,375.0,M,M,0.0,51.54612,-0.07513,51.51947,-0.05975,0.406737,0.913545,0.207912,0.978148,0.994522,-0.104528,-0.913545,0.406737,0.406737,0.913545,0.207912,0.978148,0.978148,-0.207912,5.665539e-16,-1.000000
3,22218000,EK99,BH,Y,375.0,M,M,0.0,51.54612,-0.07513,51.54612,-0.07513,0.406737,0.913545,0.207912,0.978148,0.994522,-0.104528,-0.994522,-0.104528,0.406737,0.913545,0.207912,0.978148,0.978148,-0.207912,5.000000e-01,0.866025
4,22218000,EK99,BH,Y,375.0,M,M,0.0,51.54612,-0.07513,51.54612,-0.07513,0.406737,0.913545,0.207912,0.978148,0.994522,-0.104528,-0.809017,0.587785,0.406737,0.913545,0.207912,0.978148,0.978148,-0.207912,-4.067366e-01,-0.913545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493246,22214000,EK01,WD,Y,378.0,R,M,5.0,51.55548,-0.15136,51.53250,-0.24454,0.500000,0.866025,0.587785,-0.809017,0.743145,-0.669131,0.809017,-0.587785,0.500000,0.866025,0.587785,-0.809017,0.669131,-0.743145,6.691306e-01,-0.743145
493247,22204000,EK01,WD,Y,378.0,R,M,4.0,51.52652,-0.23569,51.53250,-0.24454,0.500000,0.866025,0.587785,-0.809017,0.743145,-0.669131,-0.913545,0.406737,0.500000,0.866025,0.587785,-0.809017,0.669131,-0.743145,-5.877853e-01,0.809017
493248,22214000,EK01,WD,Y,378.0,R,M,7.0,51.49141,-0.27553,51.52274,-0.25487,0.500000,0.866025,0.587785,-0.809017,0.743145,-0.669131,-0.978148,0.207912,0.500000,0.866025,0.587785,-0.809017,0.669131,-0.743145,-8.660254e-01,0.500000
493249,22214000,EK01,WD,Y,378.0,R,M,4.0,51.49141,-0.27553,51.52274,-0.25487,0.500000,0.866025,0.587785,-0.809017,0.669131,-0.743145,0.978148,-0.207912,0.500000,0.866025,0.587785,-0.809017,0.669131,-0.743145,-4.067366e-01,-0.913545


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 493251 entries, 0 to 493250
Data columns (total 28 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   TRAIN_SERVICE_CODE_AFFECTED  493251 non-null  int64  
 1   SERVICE_GROUP_CODE_AFFECTED  493251 non-null  object 
 2   ENGLISH_DAY_TYPE             493251 non-null  object 
 3   APP_TIMETABLE_FLAG_AFF       493251 non-null  object 
 4   UNIT_CLASS_AFFECTED          493251 non-null  float64
 5   INCIDENT_REASON              493251 non-null  object 
 6   PERFORMANCE_EVENT_CODE       493251 non-null  object 
 7   PFPI_MINUTES                 493251 non-null  float64
 8   Lat_OR                       493251 non-null  float64
 9   Lon_OR                       493251 non-null  float64
 10  Lat_DES                      493251 non-null  float64
 11  Lon_DES                      493251 non-null  float64
 12  ORIG_MONTH_SIN               493251 non-null  float64
 13 

In [6]:
# num_transformer = MinMaxScaler()

# cat_transformer = OneHotEncoder(handle_unknown='ignore', drop='if_binary')



# def sin_transformer(period):
#     return FunctionTransformer(lambda x: np.sin(x / period * 2 * np.pi))


# def cos_transformer(period):
#     return FunctionTransformer(lambda x: np.cos(x / period * 2 * np.pi))


# preprocessor = ColumnTransformer([
#     ('num_transformer', num_transformer, ['Lat_OR', 'Lon_OR', 'Lat_DES', 'Lon_DES']),
#     ('cat_transformer', cat_transformer, ['ENGLISH_DAY_TYPE', 'SERVICE_GROUP_CODE_AFFECTED', 'INCIDENT_REASON',
#                                                         'UNIT_CLASS_AFFECTED', 'TRAIN_SERVICE_CODE_AFFECTED', 
#                                                      'PERFORMANCE_EVENT_CODE', 
#                                                      'APP_TIMETABLE_FLAG_AFF']),
#     ('month_sin', sin_transformer(12), ['ORIG_MONTH', 'DEST_MONTH']),
#     ('month_cos', cos_transformer(12), ['ORIG_MONTH', 'DEST_MONTH']),
#     ('day_sin', sin_transformer(31), ['ORIG_DAY', 'DEST_DAY']),
#     ('day_cos', cos_transformer(31), ['ORIG_DAY', 'DEST_DAY']),
#     ('hour_sin', sin_transformer(24), ['ORIG_HOUR', 'DEST_HOUR']),
#     ('hour_cos', cos_transformer(24), ['ORIG_HOUR', 'DEST_HOUR']),
#     ('minute_sin', sin_transformer(60), ['ORIG_MINUTE', 'DSET_MINUTE']),
#     ('minute_cos', cos_transformer(60), ['ORIG_MINUTE', 'DEST_MINUTE'])
# ], sparse_threshold=0, remainder='drop')

In [7]:
num_transformer = MinMaxScaler()

cat_transformer = OneHotEncoder(handle_unknown='ignore', sparse = False)

transformer = make_column_transformer((num_transformer, ['Lat_OR','Lon_OR', 'Lat_DES','Lon_DES']),
                                  (cat_transformer, ['ENGLISH_DAY_TYPE', 'SERVICE_GROUP_CODE_AFFECTED', 'INCIDENT_REASON',
                                                        'UNIT_CLASS_AFFECTED', 'TRAIN_SERVICE_CODE_AFFECTED', 
                                                     'PERFORMANCE_EVENT_CODE', 
                                                     'APP_TIMETABLE_FLAG_AFF']),
                                remainder = 'passthrough')


pipe = Pipeline([('transformer', transformer)])

pipe

Pipeline(steps=[('transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('minmaxscaler',
                                                  MinMaxScaler(),
                                                  ['Lat_OR', 'Lon_OR',
                                                   'Lat_DES', 'Lon_DES']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  ['ENGLISH_DAY_TYPE',
                                                   'SERVICE_GROUP_CODE_AFFECTED',
                                                   'INCIDENT_REASON',
                                                   'UNIT_CLASS_AFFECTED',
                                                   'TRAIN_SERVICE_CODE_AFFECTED',
                                                   'PERFORMANCE_EVENT_CODE',
                                                   'APP_TIMETABLE_FLAG_AFF'])]))])

In [8]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns='PFPI_MINUTES'), df['PFPI_MINUTES'], test_size=0.3)

In [9]:
X_train_scaled = pipe.fit_transform(X_train)
X_test_scaled = pipe.transform(X_test)


In [10]:
X_train_scaled[0]

array([ 0.50655971,  0.5413115 ,  0.52213911,  0.45880561,  0.        ,
        0.        ,  0.        ,  0.        ,  1.        ,  1.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        1.        ,  0.        ,  0.        ,  1.        ,  0.10452846,
        0.9945219 ,  0.74314483,  0.66913061,  0.80901699, -0.58778525,
        0.91354546, -0.40673664,  0.10452846,  0.9945219 ,  0.74314483,
        0.66913061,  0.74314483, -0.66913061,  0.80901699, -0.58

In [11]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

model_lin = LinearRegression()

cross_val_score(model_lin, X_train_scaled, y_train, cv=10, scoring='neg_root_mean_squared_error').mean()

-4.811597719136753

In [12]:
model_lin.fit(X_train_scaled, y_train)
model_lin.score(X_test_scaled, y_test)

0.13767465431135817

In [18]:
model_xgb = XGBRegressor(max_depth=10, n_estimators=500, learning_rate=0.01, n_jobs = -1)
cross_val_score(model_xgb, X_train_scaled, y_train, cv=10, scoring='neg_root_mean_squared_error').mean()

-4.217196101654763

In [19]:
model_xgb.fit(X_train_scaled, y_train)
model_xgb.score(X_test_scaled, y_test)

0.301517791919063

In [15]:
automl = AutoML()
settings = {
    "time_budget": 1800,  # total running time in seconds
    "metric": 'rmse',  # primary metrics for regression can be chosen from: ['mae','mse','r2']
    "estimator_list": ['xgboost'],  # list of ML learners; we tune XGBoost in this example
    "task": 'regression',  # task type
    "log_file_name": 'train_delays.log',  # flaml log file
    "seed": 37,  # random seed
}
automl.fit(X_train=X_train_scaled, y_train=y_train, **settings)


[flaml.automl.logger: 12-05 14:44:27] {1679} INFO - task = regression
[flaml.automl.logger: 12-05 14:44:27] {1690} INFO - Evaluation method: holdout
[flaml.automl.logger: 12-05 14:44:28] {1788} INFO - Minimizing error metric: rmse
[flaml.automl.logger: 12-05 14:44:28] {1900} INFO - List of ML learners in AutoML Run: ['xgboost']
[flaml.automl.logger: 12-05 14:44:28] {2218} INFO - iteration 0, current learner xgboost
[flaml.automl.logger: 12-05 14:44:28] {2344} INFO - Estimated sufficient time budget=16445s. Estimated necessary time budget=16s.
[flaml.automl.logger: 12-05 14:44:28] {2391} INFO -  at 0.3s,	estimator xgboost's best error=5.3670,	best estimator xgboost's best error=5.3670
[flaml.automl.logger: 12-05 14:44:28] {2218} INFO - iteration 1, current learner xgboost
[flaml.automl.logger: 12-05 14:44:28] {2391} INFO -  at 0.4s,	estimator xgboost's best error=4.8958,	best estimator xgboost's best error=4.8958
[flaml.automl.logger: 12-05 14:44:28] {2218} INFO - iteration 2, current l

[flaml.automl.logger: 12-05 14:45:11] {2391} INFO -  at 44.0s,	estimator xgboost's best error=3.9954,	best estimator xgboost's best error=3.9954
[flaml.automl.logger: 12-05 14:45:11] {2218} INFO - iteration 34, current learner xgboost
[flaml.automl.logger: 12-05 14:45:37] {2391} INFO -  at 69.5s,	estimator xgboost's best error=3.9954,	best estimator xgboost's best error=3.9954
[flaml.automl.logger: 12-05 14:45:37] {2218} INFO - iteration 35, current learner xgboost
[flaml.automl.logger: 12-05 14:45:55] {2391} INFO -  at 87.3s,	estimator xgboost's best error=3.9954,	best estimator xgboost's best error=3.9954
[flaml.automl.logger: 12-05 14:45:55] {2218} INFO - iteration 36, current learner xgboost
[flaml.automl.logger: 12-05 14:45:58] {2391} INFO -  at 90.4s,	estimator xgboost's best error=3.9954,	best estimator xgboost's best error=3.9954
[flaml.automl.logger: 12-05 14:45:58] {2218} INFO - iteration 37, current learner xgboost
[flaml.automl.logger: 12-05 14:46:06] {2391} INFO -  at 98.5

[flaml.automl.logger: 12-05 14:52:23] {2391} INFO -  at 475.1s,	estimator xgboost's best error=3.9199,	best estimator xgboost's best error=3.9199
[flaml.automl.logger: 12-05 14:52:23] {2218} INFO - iteration 69, current learner xgboost
[flaml.automl.logger: 12-05 14:52:30] {2391} INFO -  at 482.1s,	estimator xgboost's best error=3.9199,	best estimator xgboost's best error=3.9199
[flaml.automl.logger: 12-05 14:52:30] {2218} INFO - iteration 70, current learner xgboost
[flaml.automl.logger: 12-05 14:52:42] {2391} INFO -  at 494.8s,	estimator xgboost's best error=3.9122,	best estimator xgboost's best error=3.9122
[flaml.automl.logger: 12-05 14:52:42] {2218} INFO - iteration 71, current learner xgboost
[flaml.automl.logger: 12-05 14:53:23] {2391} INFO -  at 535.8s,	estimator xgboost's best error=3.9122,	best estimator xgboost's best error=3.9122
[flaml.automl.logger: 12-05 14:53:23] {2218} INFO - iteration 72, current learner xgboost
[flaml.automl.logger: 12-05 14:53:28] {2391} INFO -  at 

[flaml.automl.logger: 12-05 15:05:33] {2391} INFO -  at 1265.6s,	estimator xgboost's best error=3.9122,	best estimator xgboost's best error=3.9122
[flaml.automl.logger: 12-05 15:05:33] {2218} INFO - iteration 104, current learner xgboost
[flaml.automl.logger: 12-05 15:05:39] {2391} INFO -  at 1271.3s,	estimator xgboost's best error=3.9122,	best estimator xgboost's best error=3.9122
[flaml.automl.logger: 12-05 15:05:39] {2218} INFO - iteration 105, current learner xgboost
[flaml.automl.logger: 12-05 15:06:05] {2391} INFO -  at 1297.8s,	estimator xgboost's best error=3.9122,	best estimator xgboost's best error=3.9122
[flaml.automl.logger: 12-05 15:06:05] {2218} INFO - iteration 106, current learner xgboost
[flaml.automl.logger: 12-05 15:06:12] {2391} INFO -  at 1304.9s,	estimator xgboost's best error=3.9122,	best estimator xgboost's best error=3.9122
[flaml.automl.logger: 12-05 15:06:12] {2218} INFO - iteration 107, current learner xgboost
[flaml.automl.logger: 12-05 15:06:19] {2391} INF

In [19]:
from tensorflow.keras import Sequential, layers
nn = Sequential()
nn.add(layers.Dense(70, input_dim = X_train_scaled.shape[1], activation='relu')) 
nn.add(layers.Dense(140, activation='relu'))
nn.add(layers.Dense(50, activation='relu'))
nn.add(layers.Dense(1, activation='linear'))


In [20]:
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 70)                4970      
                                                                 
 dense_5 (Dense)             (None, 140)               9940      
                                                                 
 dense_6 (Dense)             (None, 50)                7050      
                                                                 
 dense_7 (Dense)             (None, 1)                 51        
                                                                 
Total params: 22,011
Trainable params: 22,011
Non-trainable params: 0
_________________________________________________________________


In [22]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=5, restore_best_weights=True)

nn.compile(loss='mse', 
              optimizer='adam', 
              metrics=['mae'])

history = nn.fit(X_train_scaled, y_train,
          validation_split=0.2, 
          batch_size=32,
          epochs=1000,
          callbacks=[es])

Epoch 1/1000


2023-12-05 15:50:05.485561: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 77341600 exceeds 10% of free system memory.


8632/8632 [==============================] - 24s 3ms/step - loss: 22.4926 - mae: 1.8874 - val_loss: 22.3038 - val_mae: 1.8248
Epoch 2/1000
8632/8632 [==============================] - 27s 3ms/step - loss: 21.3211 - mae: 1.8355 - val_loss: 20.7805 - val_mae: 1.8060
Epoch 3/1000
8632/8632 [==============================] - 21s 2ms/step - loss: 20.9541 - mae: 1.8267 - val_loss: 20.4158 - val_mae: 1.7238
Epoch 4/1000
8632/8632 [==============================] - 18s 2ms/step - loss: 20.3819 - mae: 1.8151 - val_loss: 20.3121 - val_mae: 1.7777
Epoch 5/1000
8632/8632 [==============================] - 21s 2ms/step - loss: 19.9943 - mae: 1.8082 - val_loss: 23.3725 - val_mae: 1.9901
Epoch 6/1000
8632/8632 [==============================] - 18s 2ms/step - loss: 19.9306 - mae: 1.8028 - val_loss: 22.5511 - val_mae: 1.8183
Epoch 7/1000
8632/8632 [==============================] - 19s 2ms/step - loss: 19.4533 - mae: 1.7980 - val_loss: 22.4087 - val_mae: 1.8349
Epoch 8/1000
8632/8632 [================

In [25]:
y_test

291059     2.0
26152      3.0
58725      0.0
364447     4.0
92290     10.0
          ... 
471089     1.0
358705     3.0
467571     1.0
263639     1.5
67847      4.0
Name: PFPI_MINUTES, Length: 147976, dtype: float64

In [26]:
y_pred = nn.predict(X_test_scaled)

  12/4625 [..............................] - ETA: 21s 

2023-12-05 15:56:09.294173: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 41433280 exceeds 10% of free system memory.


4625/4625 [==============================] - 5s 1ms/step


In [28]:
from sklearn.metrics import mean_absolute_error, max_error

max_error(y_true=y_test, y_pred=y_pred)

270.26743030548096